This uses the current head of [@ljwolf's pdio branch](https://github.com/ljwolf/pysal/tree/pdio). 

There are a few simple things we cna do to help our API become a little simpler. This is in absence of using multiple-dispatch. 

So, to show the new polymorphic weights constructors in action, I'd like to show a decorator I'd like to place in `common`, the `intercept_filepath` decorator.

In [1]:
import pysal as ps

In [2]:
datapath = ps.examples.get_path('south.shp')

In [3]:
df = ps.pdio.read_files(datapath)
file_handler = ps.open(datapath)

This could be used library-wide to enable what might look like multiple dispatch at first, but is, in fact, a simple way to preparse arguments to match the API of the defined function. 

In short, the `intercept_filepath` decorator intercepts the arguments that are passed to its function. First, it checks the type of the function's first argument. If it's a string, it attempts to open it using `pysal.open` before sending it to the original function. 

In [4]:
from pysal.common import intercept_filepath

To show, this is a simple function that expcets a geometry collection:

In [5]:
def a(coll, k):
    """
    Print the first k centroids from a geometry collection.
    
    Arguments
    -----------
    coll    : iterable
              a collection of geometries with centroids
    k       : int
              the number of centroids to print from the start of col
    """
    print([coll[i].centroid for i in range(k)])

Undecorated, it will not work if a string is passed:

In [6]:
#a(datapath, 2) #I will fail if you uncomment me

But, will work if a file handler or dataframe column is passed:

In [7]:
a(df.geometry, 1)
a(file_handler, 4)

[(-80.578701723812, 40.520355179265735)]
[(-80.578701723812, 40.520355179265735), (-80.57987548953308, 40.27464887574844), (-80.62260358646088, 40.09987235391358), (-80.66606152758811, 39.86106340922978)]


But, if we decorate the function, we can preprocess the arugments:

In [8]:
b = intercept_filepath(a) # decorators are just functions that return functions

In [9]:
print(datapath)
b(datapath, 1)

pysal/examples/south/south.shp
[(-80.578701723812, 40.520355179265735)]


Internally, `intercept_filepath` looks like:

```python
def intercept_filepath(f):
    """
    Intercept the first argument of a function if it looks like a string path
    """
    @wraps(f)
    def wrapped(*args, **kwargs):
        iargs = iter(args)
        first = next(iargs)
        rest = list(iargs)
        if isinstance(first, str):
            first = popen(first)
        return f(first, *rest, **kwargs)
    return wrapped
```

Since `intercept_filepath` uses `functools.wraps`, we also get free transformation of the docstrings. The signatures will be translated on Python 3.4 and up, but will not be translated in Python 2.7. If this is a problem, we can always document docstrings with args & kwargs separately, so that users can see which are required if they're in Python 2.7.

In [10]:
a?

In [11]:
b?

Using this, we can define a target API that *only* constructs weights from iterables or does regression from numpy arrays. Then, if we decorate that function, we can preparse its arguments to get them in the form the API expects.

This is a much cleaner way to do the dispatch-table argument parsing, since we can centralize the dispatch table's elements into one decorator. 

For example, if our dispatch/preparse table had something like:
```python
def foo(x):
    elif isinstance(x, str):
        x = open(x)
    do_stuff(x)
```
we could make it look instead like:
```python
@intercept_filepath
def foo(x):
    do_stuff(x)
```

The most obvious ways this seems to help us is in removing the various `*_from_array`, as well as allowing us to provide spreg-style userchecks to *any* function without cluttering the body of the function. 


So, in future, this could mean that any functions that we want common things done to preparse or validate input can be done like:
```python
@intercept_filepath
@intercept_dataframe
@validate_iterable(Polygon)
@enforce_shapelimit(10000)
def Rook(polygon_iterable, transformation='r', **kw):
    return do_weights()
```
This lets you clearly separate the body of the function from the preprocessing and also lets us centralize the usercheck code. 

This could also work for enforcing consistent array shaping/casting rules, but I will be investigating this going foward. 

### Right now

This means that the core of the polymorphic weights stuff is already done:

In [12]:
from pysal.weights import user2 as newW

In [13]:
rW0 = ps.rook_from_shapefile(datapath)
rW0named = ps.rook_from_shapefile(datapath, idVariable='FIPS')
rW0namedsparse = ps.rook_from_shapefile(datapath, idVariable='FIPS', sparse=True)
rW1 = newW.Rook(df)
rW2 = newW.Rook(file_handler)
rW3 = newW.Rook(datapath)
rW4 = newW.Rook(datapath, idVariable='FIPS')
rW4 = newW.Rook(datapath, idVariable='FIPS', sparse=True)

All that's needed from here to ensure that stuff like `idVariable` and `sparse` options unique to the `*_from_shapefile` functions are respected. 

In [14]:
def compare_weights(w0, *ws):
    """
    Compare any number of spatial weights objects for equality.
    
    Arguments
    ---------
    *ws      : positional arguments
               an arbitrary number of spatial weights objects
    
    Returns
    -------
    list of bools denoting whether a weight is 
    equivalent to the first argument. 
    
    Note
    ----
    Since equality is transitive, this implies all weights whose results
    are True are equivalent to each other. But, each False result
    is not necessarily equivalent to other False results. 
    """
    passes = [True]
    for wi in ws:
        this_passes = []
        w0_ordered = not all([isinstance(x, int) for x in w0.id_order])
        wi_ordered = not all([isinstance(x, int) for x in wi.id_order])
        if w0_ordered and not wi_ordered:
            for wi_idx in wi.id_order:
                w0_idx = w0.id_order[wi_idx]
                w0_weights = w0[w0_idx]
                wi_weights = wi[wi_idx]

                #de-name
                for key in w0_weights.keys():
                    name_idx = w0.id_order.index(key)
                    w0_weights[name_idx] = w0_weights[key]
                    del w0_weights[key]
                this_passes.append(w0_weights == wi_weights)
        elif wi_ordered and not w0_ordered:
            this_passes.append(compare_weights(wi, w0))
        elif (not w0_ordered) and (not wi_ordered):
            for i in w0.id_order:
                this_passes.append(w0[i] == wi[i])
        else:
            this_passes.append(None)
        passes.append(all(this_passes))
    if len(passes) == 1:
        passes = passes[0]
    return passes

In [15]:
compare_weights(rW0, rW0named, rW2, rW3, rW4)

[True, True, True, True, True]